<a href="https://colab.research.google.com/github/basketbob/ML/blob/main/netology_DS/homework7_spark_KuzinVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [ ]:
!tar -xvf spark-3.3.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.csv('iris.CSV', inferSchema=True, header=True)

In [ ]:
df.show(5)

+------------+-----------+------------+-----------+-------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|
+------------+-----------+------------+-----------+-------+-----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|
+------------+-----------+------------+-----------+-------+-----------+
only showing top 5 rows



In [ ]:
# +++При помощи VectorAssembler преобразовать все колонки с признаками в одну
# Разбить данные на train и test
# Создать модель логистической регресии или модель дерева и обучить ее
# Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве
#df_group = df.select('variety', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width')

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], outputCol='Features')

In [ ]:
df_features = assembler.transform(df)

In [ ]:
df_features.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
only showing top 5 rows



In [ ]:
train, test = df_features.randomSplit([0.8, 0.2], seed=67890)

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'Features', labelCol = 'variety_num')
lrModel = lr.fit(train)

In [ ]:
train_res = lrModel.transform(train)
test_res = lrModel.transform(test)

In [ ]:
train_res.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+-------+-----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1| Setosa|          0|[4.3,3.0,1.1,0.1]|[62.4078604302754...|[1.0,2.4543290046...|       0.0|
|         4.4|        3.0|         1.3|        0.2| Setosa|          0|[4.4,3.0,1.3,0.2]|[56.9827258033724...|[1.0,1.9354644664...|       0.0|
|         4.5|        2.3|         1.3|        0.3| Setosa|          0|[4.5,2.3,1.3,0.3]|[27.7439850534320...|[0.99999999373676...|       0.0|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|[4.6,3.1,1.5,0.2]|[56.9926754978437...|[1.0,1.2389579983...|       0.0|

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

ev = MulticlassClassificationEvaluator(labelCol='variety_num')

In [ ]:
ev.evaluate(train_res)

0.9842519685039369

In [ ]:
ev.evaluate(test_res)

0.9558242269239712